In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
from datasets import list_datasets, load_dataset
from pprint import pprint
import random
import keras
import os

In [ ]:
dataset = load_dataset('squad') ## downloading squad dataset

Found cached dataset squad (C:/Users/user/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
train_squad = pd.DataFrame(dataset["train"])
test_squad = pd.DataFrame(dataset["validation"])

In [ ]:
train_squad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87599 entries, 0 to 87598
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        87599 non-null  object
 1   title     87599 non-null  object
 2   context   87599 non-null  object
 3   question  87599 non-null  object
 4   answers   87599 non-null  object
dtypes: object(5)
memory usage: 3.3+ MB


In [ ]:
train_squad.head()
## we dont need id gonna drop it

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [ ]:
train_squad.drop("id",axis=1,inplace=True)
test_squad.drop("id",axis=1,inplace=True)

In [ ]:
train_squad["title"].value_counts()

New_York_City            817
American_Idol            802
Beyoncé                  758
Frédéric_Chopin          697
Queen_Victoria           680
                        ... 
Great_Plains              47
Tristan_da_Cunha          44
Pitch_(music)             36
Matter                    24
Myocardial_infarction     22
Name: title, Length: 442, dtype: int64

In [ ]:
avg_words_context = round(sum([len(i.split()) for i in train_squad["context"]])/len(train_squad))
avg_words_question = round(sum([len(i.split()) for i in train_squad["question"]])/len(train_squad))
avg_words_context,avg_words_question

(120, 10)

In [ ]:
max_words_context = (max([len(i.split()) for i in train_squad["context"]]))
max_words_question = (max([len(i.split()) for i in train_squad["question"]]))
max_words_context,max_words_question

(653, 40)

In [ ]:
(653*0.1 + 120*1.9) / 2

146.65

## Tokenization

In [ ]:
encoder_text = train_squad["context"][:4000]
decoder_text = train_squad["question"][:4000]

In [ ]:
from keras.preprocessing.text import Tokenizer

def token_fit(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    dictionary = tokenizer.word_index
    
    word2i ={k:v for (k,v) in dictionary.items()}
    i2word = {v:k for (k,v) in word2i.items()}
    
    return tokenizer, word2i,i2word


In [ ]:
tokenizer,word2i,i2word = token_fit(encoder_text+decoder_text)

In [ ]:
print("Most occured 10 words in vocab")
sorted(tokenizer.word_counts,key=tokenizer.word_counts.get,reverse=True)[:10]

Most occured 10 words in vocab


['the', 'of', 'in', 'and', 'to', 'a', 'was', 'for', 'on', 'as']

In [ ]:
vocab_size= len(word2i)
vocab_size

12113

In [ ]:
def text2sequence(context,question,tokenizer):
    encoder_seq = tokenizer.texts_to_sequences(context)
    decoder_seq = tokenizer.texts_to_sequences(question)
    
    return encoder_seq,decoder_seq

In [ ]:
encoder_seq,decoder_seq = text2sequence(encoder_text,decoder_text,tokenizer)

In [ ]:
print("Original Text \n-----------\n",encoder_text[0])
print("Turned into sequences \n-----------\n",encoder_seq[0])

Original Text 
-----------
 Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Turned into sequences 
-----------
 [8672, 1, 97, 35, 6, 556, 658, 779, 1, 274, 8673, 1148, 2376, 16, 6, 1170, 2451, 2, 1, 3917, 1400, 1926, 3, 3329, 2, 1, 274, 258, 4, 3454, 23, 16, 6, 5041, 2451, 2, 5778, 12, 5779, 8674, 12, 1, 702, 8675, 8676, 739, 8677, 599, 5, 1, 274, 25

In [ ]:

max_len_enc = 0
max_len_dec = 0

for enc,dec in zip(encoder_seq,decoder_seq):
    if len(enc) > max_len_enc:
        max_len_enc = len(enc)
    if len(dec) > max_len_dec:
        max_len_dec= len(dec)

In [ ]:
max_len_enc,max_len_dec

(518, 29)

In [ ]:
from keras.utils import pad_sequences
encoder_data = pad_sequences(encoder_seq,maxlen=max_len_enc,dtype="int32",padding="post",truncating="post")
decoder_data = pad_sequences(decoder_seq,maxlen=max_len_dec,dtype="int32",padding="post",truncating="post")

#### as we can see it added extra zeros to make them same size(we selected post to add zeros after)

In [ ]:
encoder_data[0][40:200]

array([  12,    1,  702, 8675, 8676,  739, 8677,  599,    5,    1,  274,
        258,   16,    1, 2531,    2,    1, 4684, 2076, 1926,  907,    1,
       2531,   16,    1, 2618,    6, 8678,  351,    2, 4117,    4, 8679,
         23,   16,    6, 5780,    2,    1, 2618,   14, 3739,  606,   70,
          1, 3917, 1400, 8680,  480,    5, 2532, 8681, 8682,    3, 7988,
         14,    1,  327,    2,    1,  274, 1548,    4,    3,    6, 1081,
        233,   11, 6389,  206,  158, 4118,    4,    1, 1148, 2376,   16,
          6, 2182,  499, 2131, 2451,    2, 1400,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

### Pretrained embedding using glove

In [ ]:
path_to_glove_file = os.path.join(
 "./glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
num_tokens = vocab_size + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))

In [ ]:
for word, i in word2i.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [ ]:
print("Converted %d words (%d misses)" % (hits, misses))

Converted 11076 words (1037 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
num_samples = len(encoder_seq)
num_samples

4000

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 128
encoder_inputs = Input(shape=(None, max_len_enc))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_data)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, max_len_dec))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_data,
                                     initial_state=encoder_states)
decoder_dense = Dense(max_len_dec, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (4000, 518)